<a href="https://colab.research.google.com/github/Maplemx/Agently/blob/main/playground/workflow_series_05_seperating_generation_steps_and_reply_in_every_step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seperating Generation Steps and Reply to User in Every Step`#Agently_Workflow_Showcase_Series - 05`

## Demo Description

**Author:** Agently Team

**Series Link**:

[Last Document]: [04 - Draw a Workflow Graph to Help You Observe the Workflow](https://github.com/Maplemx/Agently/blob/main/playground/workflow_series_04_draw_a_workflow_graph.ipynb)

**Description:**

Now you have already been familiar with the concept ideas and the basic usages of **Agently Workflow**. From this showcase, we will dive deeper to discuss how to use workflow to solve problems in more complex scene.

As we all know, LLMs' abilities show different between languages. Those models which trained by raw materials in English will perform better when asked questions in English.

So maybe we can try translating user's inputs into English and get English response from LLMs then translate the English response into user's original language. To ensure the questions and answers in unison between different languages, we reply every step's result to user. That also means **user will get more than one replies step by step after inputing one question**.

This showcase will demostrate how to do that with **Agently Workflow**.

读到本文档的时候，你应该已经非常熟悉**Agently Workflow**的思想理念和基础用法了。从本案例开始，我们将更深度地讨论如何在更为复杂的实践场景中使用workflow去解决问题。

如我们所知，在使用不同的语种和模型交互时，语言模型展现出的回答能力是不一样的。那些更多使用英文素材进行训练的模型，在处理英文问题时，给出的回答可能会更加优质。

所以，或许我们可以尝试构建一个工作流，将用户输入的问题转化为英文问题提交给模型，在获取模型的英文回复后，再将这个回复翻译为用户输入问题时所用的语言。同时，为了保证工作流中，翻译前后内容的一致性，我们将在工作的每一个阶段都将模型的处理结果回复呈现给用户。也就是说，**当用户输入一个问题之后，他将获得多于一次的分阶段回复**。

这个案例将会向您演示我们如何使用**Agently Workflow**编排工作流程来做到这一点。

## Step 1: Install Packages

In [ ]:
!pip install -U -q Agently>=3.2.2 mermaid-python

## Step 2: Workflow Showcase

### Workflow Code

In [35]:
import Agently
agent_factory = (
        Agently.AgentFactory()
        .set_settings("model.Google.auth.api_key", "")
        .set_settings("current_model", "Google")
)

workflow = Agently.Workflow()
agent = agent_factory.create_agent()

# Step 1: Define Workflow Chunks
## Start Chunk
@workflow.chunk(
    chunk_id="Start",
    type = "Start"
)

## User Input Chunk
@workflow.chunk(chunk_id="User Input")
def user_input_executor(inputs, storage):
    return input("[User]: ")

## Reply Chunk
@workflow.chunk(chunk_id="Reply")
def reply_executor(inputs, storage):
    print(">>> ", inputs["input"])

## Translate To Chunk
@workflow.chunk(chunk_id="Translate To")
def translate_to_executor(inputs, storage):
    result = (
        agent
            .input(inputs["input"])
            .output({
                "original_language": ("str", "natural language name of the content in {input}"),
                "translation": ("str", "translate {input} into English"),
            })
            .start()
    )
    # save original language of user input into storage
    storage.set("original_language", result["original_language"])
    return result["translation"]

## Answer Chunk
@workflow.chunk(chunk_id="Answer")
def answer_executor(inputs, storage):
    return (
        agent
            .input(inputs["input"])
            .start()
    )

## Translate From Chunk
@workflow.chunk(chunk_id="Translate From")
def translate_from_executor(inputs, storage):
    return (
        agent
            .input(inputs["input"])
            # load original language of user input from storage
            .output(f"translate {{input}} into { storage.get('original_language') }")
            .start()
    )

# Step 2: Connect Chunks
## Main Workflow
workflow.chunks["Start"].connect_to(workflow.chunks["User Input"])
workflow.chunks["User Input"].connect_to(workflow.chunks["Translate To"])
workflow.chunks["Translate To"].connect_to(workflow.chunks["Answer"])
workflow.chunks["Answer"].connect_to(workflow.chunks["Translate From"])
## Replies
workflow.chunks["Translate To"].connect_to(workflow.chunks["Reply"])
workflow.chunks["Answer"].connect_to(workflow.chunks["Reply"])
workflow.chunks["Translate From"].connect_to(workflow.chunks["Reply"])

print("Ready to start!")

Ready to start!


### Workflow Graph

In [4]:
from mermaid import Mermaid
Mermaid(workflow.draw())

### Start

In [36]:
workflow.start()

[User]: 什么是工作流？
>>>  What is a workflow?
>>>  **Definition:**

A workflow is a sequence of tasks or steps that define a specific business process or activity. It automates the flow of work, ensuring that each task is executed in the correct order and by the intended person or system.

**Key Features:**

* **Automation:** Workflows remove manual intervention by automatically triggering tasks based on pre-defined conditions.
* **Sequence:** Tasks are arranged in a logical order, ensuring that each step is completed before the next one begins.
* **Assignment:** Workflows assign tasks to specific individuals or departments based on roles or responsibilities.
* **Tracking:** Progress can be tracked throughout the workflow, providing visibility and accountability.
* **Approval:** Workflows can include approval processes to ensure that tasks meet certain standards or criteria.

**Benefits of Workflows:**

* **Increased efficiency:** Automating tasks reduces manual labor and streamlines proces

---

[**_<font color = "red">Agent</font><font color = "blue">ly</font>_** Framework - Speed up your AI Agent Native application development](https://github.com/Maplemx/Agently)